Task:
Write a Python script that fetches data from the following fake API https://jsonplaceholder.typicode.com/posts/<post_id> for post_ids ranging from 1 to 100. The script should:

- Fetch data concurrently using Python’s concurrency mechanisms (using either concurrent.futures, asyncio, or multiprocessing).
- Write the post title for each fetched response into a file called posts.txt.
- Ensure that the file writes happen in the same order as the post_ids (even if the requests are handled concurrently).
- Handle potential exceptions or network issues gracefully, such as timeouts.

**Constraints:**
- You may choose your preferred concurrent programming tool (**concurrent.futures, asyncio, or multiprocessing**). The test will assess both feasibility (ideally solved in 10–15 minutes) and alignment with idiomatic Python.
- Ensure the solution is efficient and maintains the correct order of the data.

**Input:**
- post_ids from 1 to 100.

**Output:**
- A file named posts.txt, where each line contains a fetched post’s title. The order of lines should match the order of the post_ids.

In [5]:
# !pip install requests (if not installed)
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

In [7]:
URL_TEMPLATE = "https://jsonplaceholder.typicode.com/posts/{}"
OUTPUT_FILE = "posts.txt"

def fetch_post(post_id):
    """Fetch a single post by ID."""
    try:
        response = requests.get(URL_TEMPLATE.format(post_id), timeout=5)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
        return post_id, data["title"]
    except Exception as e:
        return post_id, f"Error: {e}"

def write_to_file(posts):
    """Write posts in order to a file."""
    with open(OUTPUT_FILE, "w") as f:
        for post_id, title in sorted(posts, key=lambda x: x[0]):
            f.write(f"{post_id}: {title}\n")

In [17]:
post_ids = range(1, 11)  # IDs 1 to 100
posts = []

with ThreadPoolExecutor(max_workers=10) as executor:  # Adjust worker threads as needed
    futures = {executor.submit(fetch_post, post_id): post_id for post_id in post_ids}

    for future in as_completed(futures):
        try:
            # get the result of the function fetch_post
            post_id, title = future.result()
            posts.append((post_id, title))
        except Exception as e:
            posts.append((post_id, e))

write_to_file(posts)